# **Table names used:**


**first_table**  :   SpecimenFieldReport.

**second_table**  :   SpecimenCompressionTestResult

**log_file**  :  Log table to store the invalid files


In [ ]:
import time
start = time.time()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pdfplumber -q
!pip install regex
!pip install camelot-py[base]
!apt-get install ghostscript
!pip uninstall -y 'PyPDF2>=3.0'
!pip install 'PyPDF2<3.0'

In [ ]:
import regex as re
import pdfplumber
import pandas as pd
import numpy as np
import os
import camelot as cam
import pandas as pd
from numpy.core.fromnumeric import transpose
from pathlib import Path
from datetime import datetime

# ***SCRAPING DATA INTO FIRST TABLE***

**Function to extract the data into First table**

In [ ]:
def extract_first_table(path, text):

  global log_file

  
  Project_Name = Mix_id = City = Date_casted = Date_Report_Issued = Weather = Temperature_Ambient = concrete_supplier = ''
  Req_load = Location = Structural_Element = Mix_id = Building_Height = Water_Added = Admixtures_noted = specimen_size = ''


  # Fetching the file name
  File_name = Path(path).stem


  for row in text.split('\n'):
    
    #City
    if Project_Name == '':
      x_proj = re.search("Proj", row)
      
      if x_proj:
        string_proj = row.replace(" ", "")   #print(string_proj)
        start = ','
        end = ','
        City = (string_proj.split(start))[1].split(end)[0]

        #Project Name                                                                    # Can it be made a default value?
        start = 'Project:'
        end = '–'
        p_name = (string_proj.split(start))[1].split(end)[0]
        Project_Name = p_name.replace('x', ' x ')

      else:
        City = Project_Name = np.nan


    #Date & Cylinder Pick-up Date:
    if Date_casted == '':
      x_d = re.search("Date:", row)
      if x_d:
        x_pick = re.search("Pick-up Date:", row)
        if x_pick:
          Cylinder_pick_up_date = row.split("Pick-up Date:",1)[1].strip()
        else:
          Date_casted = row.split("Date:",1)[1].strip()
      else:
        Date_casted == ''


    #Date Report Issued & Date tested
    x_date = re.search("Date", row)
    x_D = re.search("DATE", row)
    if x_date:
      x_rep = re.search("Date Report Issued:", row)
      if x_rep:
        Date_Report_Issued = row.split("Date Report Issued:",1)[1].strip()
  
    elif x_D:
      Date_Tested = row.split("DATE TESTED",1)[1].strip()


    #Weather
    if Weather == '':
      x_w = re.search("Weather", row)
      if x_w:
        string_weather = row.replace(" ", "")   #print(string_proj)
        start = 'Weather:'
        end = 'Air'
        Weather = (string_weather.split(start))[1].split(end)[0].strip()
      else:
        Weather = ''

  
    #Temperature_Ambient
    if Temperature_Ambient == '':
      x = re.search("Air", row)
      if x:
        string_air = row.replace(" ", "")  

        AirTemp = re.search("AirTemperature:", string_air)
        if AirTemp:
          Temperature_Ambient = string_air[AirTemp.end():][:-3]    #else: Temperatures_Ambient = ''
      else:
        Temperature_Ambient = ''


    #Building Height/Number of Floors
    if Building_Height == '':
      x_bld_ht = re.search("Building height", row)
      if x_bld_ht:
        Building_Height = "Building Height is found"
      else:
        Building_Height = np.nan


    #Structural Element
    if Structural_Element == '':
      x_str_el = re.search("Structural", row)
      if x_str_el:
        Structural_Element = "Structural Element is found"
      else:
        Structural_Element = np.nan


    #Location of Placement
    if Location == '':
      x_loc = re.search("Locations:", row)
      if x_loc:
        Location = row.split("Locations:",1)[1].strip()            
      else:
        Location = ''

    
    #Mix ID
    if Mix_id == '':
      x_mix = re.search("Mix", row)
      if x_mix:
        string_mix = row.replace(" ", "")
        start = 'Mix'
        end = '.'
        mixid = (string_mix.split(start))[1].split(end)[0]
        Mix_id = re.sub('\D', '', mixid)
      else:
        Mix_id = ''


    #Water Added
    if Water_Added == '':
      x_water = re.search("Water", row, re.IGNORECASE)
      if x_water:
        Water_Added = "Water found"
      else:
        Water_Added = ''

    
    #Any Admixtures noted, 
    if Admixtures_noted == '': 
      x_ad_mix = re.search("Admix", row, re.IGNORECASE)

      if x_ad_mix:
        Admixtures_noted = "Admixtures noted is found"
      else:
        Admixtures_noted = np.nan


    #Required Load
    if Req_load == '':
      x = re.search("CLASS", row)
      if x:
        string_req_load = row.replace(" ", "")  
        start = 'CLASS(PSI)'
        end = 'Max/Min'
        Req_load = (string_req_load.split(start))[1].split(end)[0].strip()

      else:
        Req_load = ''


    #Concrete supplier
    if concrete_supplier == '':
      x = re.search("Supplier", row)
      if x:
        string_supplier = row.replace(" ", "")  
        start = 'ConcreteSupplier:'
        end = 'CylinderPick'
        concrete_supplier = (string_supplier.split(start))[1].split(end)[0].strip()
      else:
        concrete_supplier = ''
    
 
    #Specimen size
    if specimen_size == '':
      x = re.search("Field Set", row)
  
      if x:
        string_size = row.replace(" ", "")  
        start = 'FieldSet('
        end = ')'
        size = (string_size.split(start))[1].split(end)[0].strip().replace('"', "")
        specimen_size = re.sub("[^A-Z0-9]", "", size,0,re.IGNORECASE)
      else:
        specimen_size = ''



  #### IMPORTING FIELDS TO A DATA FRAME ####
  Table_1_header = ["ReportFileName", "Project_Name", "City", "Date_Molded", "ReportDateIssued", "SiteWeather", "SiteTemperature", "ConcreteSupplier", "BatchRequiredStrength", 
                  "ConcretePlacementLocation", "Structural_Element", "BatchMixID", "Building_Height", "BatchWaterAdded", "Admixtures_noted", "BatchSpecimenSize" ]

  Values = [File_name, Project_Name, City, Date_casted, Date_Report_Issued, Weather, Temperature_Ambient, concrete_supplier, Req_load, 
         Location, Structural_Element, Mix_id, Building_Height, Water_Added, Admixtures_noted, specimen_size]

  new_row_table_1 = pd.DataFrame(Values, Table_1_header).T

  

  ##############  Concrete Field Inspection and Test Results  ################


  all_tables = cam.read_pdf(path, pages = 'all', flavor = 'lattice', copy_text=['h'])

  x_table_field = []

  invalid_time_log = []


  for i in range(all_tables.n):

    if 0 < i <= repeated:
      
      tab = all_tables[i].df

      #Drop unwanted columns:
      tab_new = tab.drop(labels=[1,2,3], axis=0)  

      #Converting the dataframe to string
      tab_new = tab_new.astype(str)

      # Replacing empty spaces with Null Values
      tab_new = tab_new.replace(r'^\s*$', np.nan, regex=True)

      # Dropping the columns with all Null values:
      tab_new = tab_new.dropna(axis=1, how='all')

      # Transpose
      tab_tran = tab_new.T

      # Converting first row into header:
      headers = tab_tran.iloc[0]
      tab_field = pd.DataFrame(tab_tran.values[1:], columns = headers) 

      tab_field['Field Set (4”x8”) #'] = tab_field['Field Set (4”x8”) #'].str.replace(r'[^\d.]+', '')

      # Replacing the names of first column elements to 'Lab_No' and 'Density' fields:
      df_Table_2 =  tab_field
      df_Table_2 = df_Table_2.rename(columns={'Field Set (4”x8”) #': 'Lab_no'})
      df_Table_2 = df_Table_2.rename(columns={'Density (lbs/cu ft)': 'Unit_Weight (lbs/cu ft)'}) 


      # Chaging the format of 'Time Discharge' and making the values NULL, for invalid time formats
      # And storing them in logfiles.
      df_Table_2['Time Discharge'] = df_Table_2['Time Discharge'].str.replace('\D', '', regex=True)     # Removind values other than digits
      
      for index in range(len(df_Table_2)):

        time_dis = df_Table_2.loc[index, 'Time Discharge']
        
        if len(time_dis) == 4:
          time_conv = datetime.strptime(time_dis, '%H%M').time()
          t = time_conv.strftime('%H:%M:%S')
          t1 = time_conv.strftime('%I:%M %p')
          df_Table_2.loc[index,'Time Discharge'] = t1
          df_Table_2.loc[index, 'Batch_Time_Molded'] = t

        else:
          lab_number = df_Table_2.loc[index,'Lab_no']
          df_Table_2.loc[index,'Time Discharge'] = ''                                                                                     
          df_Table_2.loc[index, 'Batch_Time_Molded'] =''                                                                                  
          Comment = 'Passed NULL value for Time molded for the lab no', lab_number, ' due to incorrect time format'
          log_file = log_file.append({'File Name' : File_name, 'Comment' : Comment}, ignore_index = True)


      # Rnaming the columns
      df_Table_2.rename(columns={"Air Content (%)": "BatchAirContent"}, inplace=True)
      df_Table_2.rename(columns={"Slump (inches)": "BatchtSlump"}, inplace=True)
      df_Table_2.rename(columns={"Unit_Weight (lbs/cu ft)": "BatchUnitWeight"}, inplace=True)
      df_Table_2.rename(columns={"Concrete Temperature (°F)": "BatchTemperature"}, inplace=True)
      df_Table_2.rename(columns={"Time Discharge": "Time_Molded"}, inplace=True)


      if i == 1:
        x_table_field = df_Table_2
      else:
        x_table_field = pd.concat([x_table_field, df_Table_2], axis=0, ignore_index=True)

    else:
      continue


  # MERGING BOTH THE TABLES
  
  tables_merged = pd.concat([new_row_table_1, x_table_field], axis=1)

  tables_merged["ReportFileName"] = File_name 
  tables_merged["Project_Name"] = Project_Name
  tables_merged["City"] = City
  tables_merged["Date_Molded"] = Date_casted
  tables_merged["ReportDateIssued"] = Date_Report_Issued
  tables_merged["SiteWeather"] = Weather
  tables_merged["SiteTemperature"] = Temperature_Ambient
  tables_merged["ConcreteSupplier"] = concrete_supplier
  tables_merged["BatchRequiredStrength"] = Req_load
  tables_merged["ConcretePlacementLocation"] = Location
  tables_merged["Structural_Element"] = Structural_Element
  tables_merged["BatchMixID"] = Mix_id
  tables_merged["Building_Height"] = Building_Height
  tables_merged["BatchWaterAdded"] = Water_Added
  tables_merged["Admixtures_noted"] = Admixtures_noted
  tables_merged["BatchSpecimenSize"] = specimen_size

   
  # Changing the date format of 'Date_Casted' field:
  dt_cas = datetime.strptime(Date_casted, '%B %d, %Y').date()
  new_date_format = dt_cas.strftime('%Y-%m-%d')

 
  # Batch_time_molded field:

  for count in range(len(tables_merged)):
    
    if tables_merged.loc[count, 'Batch_Time_Molded'] != '':
      tables_merged.loc[count, 'Batch_Time_Molded'] = new_date_format + ' ' + tables_merged.loc[count, 'Batch_Time_Molded']

  first_table = tables_merged


  return first_table

  return log_file



# ***SCRAPING DATA INTO SECOND TABLE***

In [ ]:
def extract_second_table(path, text):


    
  read_all_tables = cam.read_pdf(path, pages = 'all', flavor = 'lattice', copy_text=['h'])


  x_table = []


  for i in range(read_all_tables.n):

    if i <= repeated:
      continue
    else:

      tab = read_all_tables[i].df

      #Converting the dataframe to string
      tab = tab.astype(str)

      # Replacing empty spaces with Null Values
      tab = tab.replace(r'^\s*$', np.nan, regex=True)

      # Dropping the columns with all Null values:
      tab = tab.dropna(axis=1, how='all')

      #Drop unwanted columns:
      tab_new = tab.drop(labels=[2,3,6,8,9,10,11], axis=0)
      tab_new_trans = tab_new.T

      # Converting first row into header and Changing name of the first header element
      headers = tab_new_trans.iloc[0]
      tab_final = pd.DataFrame(tab_new_trans.values[1:], columns = headers)
      tab_final.columns.values[0:1] =['Lab_No']

      # Changing the name of the field and converting the unit:
      tab_final = tab_final.rename(columns={'PSI': 'Unit_Load (PSI)'})

      #tab_final['Unit_Load (PSI)'] = pd.to_numeric(tab_final['Unit_Load (PSI)'])
      #tab_final['Unit_Load (PSI)'] = (tab_final['Unit_Load (PSI)'] * 0.4535).round(decimals=2)
      #tab_final = tab_final.rename(columns={'Unit_Load (PSI)': 'Unit_load (kgs/sq in)'})

      tab_final = tab_final.rename(columns={'Unit_Load (PSI)': 'Unit_load (lbs/sq in)'})
      tab_final['Lab_No'] = tab_final['Lab_No'].str.replace(r'[^\d.]+', '')

      # Dropping the rows with if any Null values:
      tab_final = tab_final.dropna()

      if i == repeated + 1:
        x_table = tab_final                                                  
      else:
        x_table = pd.concat([x_table, tab_final], axis=0, ignore_index=True)


  # Renaming the columns
  x_table.rename(columns={"CYLINDER #": "CylinderTestID"}, inplace=True)
  x_table.rename(columns={"DATE TESTED": "SpecimenDateTested"}, inplace=True)
  x_table.rename(columns={"TEST AT (days)": "SpecimentAgeTested"}, inplace=True)
  x_table.rename(columns={"Unit_load (lbs/sq in)": "SpecimenMeasuredStrength"}, inplace=True)

  return x_table


**Importing bulk data into the function:**

In [ ]:
first_table = []
log_table = []
second_table = []
time_format = ''
df_log = []
log_file = []


# Importing the file folder and reading the files.
folder_path = '/content/drive/MyDrive/Colab_Notebooks_Main/Capstone/Files_no_issue'


# Creating dataframes to store lod files:
df_log = pd.DataFrame(columns=['File Name', 'Comment'])
log_file = pd.DataFrame(columns=['File Name', 'Comment'])                 



for file in os.listdir(folder_path):
  if file.endswith(".pdf"):
    print(file)


    # Skipping the files with age days other than 7 or 28 days,  and maintaining them in Log table
    res = re.findall(r'\(.*?\)', file)
    if not '7' in str(res):
      if not '28' in str(res):

        df_log = df_log.append({'File Name' : file, 'Comment' : 'Invalid file with Age days other than 7 or 28'}, ignore_index = True)

        continue

    path = folder_path + '/' + file

    pdf = pdfplumber.open(path)
    page = pdf.pages[0]
    text = page.extract_text()



    # Cheking the number of times the string 'Field Set' is repeated, to count the number of 'Concrete Field Inspection and Test Results' tables:
    repeated = text.count("Field Set")


    # Importing data to the functions

    extracted_first = extract_first_table(path, text)

    extracted_second = extract_second_table(path, text)



    # Appending data into tables:

    first_table = extracted_first.append(first_table, ignore_index=True)

    second_table = extracted_second.append(second_table, ignore_index=True)

    log_file = pd.concat([log_file, df_log], axis=0, ignore_index=True)



    # Delteing duplicate entries from first_table (SpecimenFieldReport) with same Date_Molded and Lab_no
    new_first_table = first_table.drop_duplicates(subset = ['Date_Molded', 'Lab_no', 'Time_Molded']).reset_index(drop = True)  

    # Delteing duplicate entries from second_table (SpecimenCompressionTestResult):
    new_second_table = second_table.drop_duplicates()



In [ ]:
# Calculating the time taken for data scrapping

end = time.time()
time_taken = end - start
print('Code took ' ,end - start, ' seconds.')
print('Code took ' ,time_taken/60, ' minutes.')

# ***EXPORTING DATA TO EXCEL:***

In [ ]:
first_table.to_excel(r'C:\Users\ramgo\Desktop\Extracted data\first_table.xlsx', index=False)

In [ ]:
second_table.to_excel(r'C:\Users\ramgo\Desktop\Extracted data\second_table.xlsx', index=False)

In [ ]:
log_file.to_excel(r'C:\Users\ramgo\Desktop\Extracted data\log_table.xlsx', index=False)

In [ ]:
new_first_table.to_excel(r'C:\Users\ramgo\Desktop\Extracted data\new_first_table.xlsx', index=False)

In [ ]:
new_second_table.to_excel(r'C:\Users\ramgo\Desktop\Extracted data\new_second_table.xlsx', index=False)